In [ ]:
%pip install opencv-python numpy albumentations

In [4]:
# changeable values
DATASET_FOLDER = 'dataset_split_segmentation'

MODEL = 'efficientnetb3'
INPUT_SIZE = (512,512)
CLASSES = 3

BATCH_SIZE = 4
EPOCHS = 100 # with EarlyStopping


In [ ]:
#load dataset into numpy array
import os
import numpy as np
import cv2

def load_data(image_folder_path: str, mask_folder_path: str, tumor_type: str) -> np.ndarray:
    images = []
    masks = []
    for file in os.listdir(os.path.join(image_folder_path,tumor_type)):
        image_path = os.path.join(image_folder_path, tumor_type, file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = image.astype(np.float32) / 255
        image = np.expand_dims(image, axis=-1)

        mask_path = os.path.join(mask_folder_path, tumor_type, file)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = (mask > 127).astype(np.float32)
        mask = np.expand_dims(mask, axis=-1)

        images.append(image)
        masks.append(mask)
        
    return np.array(images), np.array(masks)

load_data('dataset_split_segmentation/test/images', 'dataset_split_segmentation/test/masks', 'glioma')



In [ ]:
#setup augmentation
import albumentations as alb

training_augmentation = alb.Compose([
    alb.HorizontalFlip(p=0.5),
    alb.Rotate(limit=25, p=0.7),
    alb.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=20,p=0.7),

    alb.ElasticTransform(alpha=0.5, sigma=30, alpha_affine=20, p=0.2),
    alb.GridDistortion(p=0.15),

    #image only
    alb.RandomBrightnessContrast(p=0.5),
    alb.RandomGamma(p=0.5),

    alb.GaussianBlur(blur_limit=(3,7), p=0.5),
    alb.GaussNoise(var_limit = (10.0,50.0), p=0.3),

    alb.CLAHE(clip_limit=4.0, p=0.5),

    alb.CoarseDropout(max_holes = 4, max_height = 16, max_width = 16, p=0.1)
])

validation_augmentation = alb.Compose([])